In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd
import selenium.common.exceptions

df_docentes = pd.read_csv("corpo_docente_limpo.csv")
orcid_list = df_docentes['ORCID'].dropna().unique().tolist()
orcid_list = [o for o in orcid_list if o and o != 'n/a']

chromedriver_path = r'C:\chromedriver\chromedriver.exe'
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service)

resultados = []

for orcid in orcid_list:
    driver.get("https://www.scopus.com/freelookup/form/author.uri")
    time.sleep(2)

    # Aceita cookies, se aparecer
    try:
        botao_cookies = driver.find_element(By.ID, "accept-cookies")
        botao_cookies.click()
        time.sleep(1)
    except selenium.common.exceptions.NoSuchElementException:
        try:
            botao_cookies = driver.find_element(By.XPATH, "//*[contains(text(),'Accept all cookies')]")
            botao_cookies.click()
            time.sleep(1)
        except:
            pass

    try:
        campo_orcid = driver.find_element(By.NAME, "orcidId")
        campo_orcid.clear()
        campo_orcid.send_keys(orcid)
        time.sleep(1)
        botao_buscar = driver.find_element(By.ID, "orcidSubmitBtn")
        botao_buscar.click()
        time.sleep(4)

        # Clica no nome do autor
        try:
            nome_autor = driver.find_element(By.XPATH, "//td[contains(@class, 'authorResultsNamesCol')]/a")
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", nome_autor)
            driver.execute_script("arguments[0].focus();", nome_autor)
            time.sleep(0.5)
            driver.execute_script("""
                var evt = document.createEvent('MouseEvents');
                evt.initMouseEvent('click',true,true,window,1,0,0,0,0,false,false,false,false,0,null);
                arguments[0].dispatchEvent(evt);
            """, nome_autor)
            time.sleep(4)
            # EXTRAÇÃO PRECISA:
            try:
                citations = driver.find_element(By.XPATH, "//span[@data-testid='unclickable-count']").text
            except:
                citations = ''
            try:
                documents = driver.find_element(By.XPATH, "(//li[contains(@class,'MetricSection-module__omEUT')][2]//span[contains(@class,'Typography-module__lVnit')])[1]").text
            except:
                documents = ''
            try:
                h_index = driver.find_element(By.XPATH, "(//li[contains(@class,'MetricSection-module__omEUT')][3]//span[contains(@class,'Typography-module__lVnit')])[1]").text
            except:
                h_index = ''
            resultados.append({
                'ORCID': orcid,
                'Documents': documents,
                'Citations': citations,
                'H-index': h_index,
                'Status': 'OK'
            })
            driver.back()
            time.sleep(2)
        except selenium.common.exceptions.NoSuchElementException:
            resultados.append({
                'ORCID': orcid,
                'Documents': '',
                'Citations': '',
                'H-index': '',
                'Status': 'Nenhum autor encontrado para esse ORCID no Scopus'
            })
            continue
        except Exception as e:
            resultados.append({
                'ORCID': orcid,
                'Documents': '',
                'Citations': '',
                'H-index': '',
                'Status': f'Erro ao clicar no nome: {e}'
            })
    except Exception as e:
        resultados.append({
            'ORCID': orcid,
            'Documents': '',
            'Citations': '',
            'H-index': '',
            'Status': f'Erro busca: {e}'
        })

driver.quit()

df = pd.DataFrame(resultados)
try:
    df.to_excel("scopus_metrics.xlsx", index=False)
    print("Arquivo scopus_metrics.xlsx gerado!")
except PermissionError:
    import datetime
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_name = f"scopus_metrics_{timestamp}.xlsx"
    df.to_excel(backup_name, index=False)
    print(f"Arquivo salvo como {backup_name} por estar aberto em outro programa!")

Arquivo scopus_metrics.xlsx gerado!
